In [1]:
# importazione delle librerie
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
from itertools import combinations, permutations

import Lib.Funtions as fn
import Lottomatica.Funtions as Lfn
import Models.Lottomatica as Lottomatica
from Lib.Path import PATH_ESTR

# importiamo il path

In [19]:
# upload arhivio
fn.update_archivio()

CREIAMO LA NOSTRA CLASSE PER IL GIOCO DEL LOTTO

In [2]:
# creiamo una lista in numpy con tutti gli ambi in 90 numeri
numeri = np.arange(1,91)
ambi = np.array(list(combinations(numeri,2)))
print(len(ambi))

4005


In [ ]:
# creiamo in dataframe contenenti tutti gli ambi e il loro numero spia a 5, 10 e 20 estrazioni
dataframe = pd.DataFrame(columns=['Ambi', 'uscite', 'Spia10', 'rip10', 'tra10','Spia20', 'rip20', 'tra20'])
n = Lottomatica.Lotto()
ruota = 'BA'
estrazioni = n.carica_estrazioni(ruota)

for i in ambi:
    i = list(map(int, i))
    #print(f"Coppia di numeri: {i}")
    n.carica_numero(i, estrazioni, 10)
    n_uscite = n.n_uscite
    spia10 = n.spia_num
    rip10 = n.spia_rip
    # calcoliamo il ritardo dell'uscita della spia (se è negativa significa che è in ritardo)
    tra10 = 10 - Lfn.calcola_uscita_da_spia(spia10, ruota, 10)
    n.carica_numero(i, estrazioni, 20)
    spia20 = n.spia_num
    rip20 = n.spia_rip
    # calcoliamo il ritardo dell'uscita della spia (se è negativa significa che è in ritardo)
    tra20 = 20 - Lfn.calcola_uscita_da_spia(spia20, ruota, 20)

    nuova_riga = pd.DataFrame({
        'Ambi': [i],
        'uscite': [n_uscite],
        'Spia10': [spia10],
        'rip10': [rip10],
        'tra10': [tra10],
        'Spia20': [spia20],
        'rip20': [rip20],
        'tra20': [tra20]
    })

    dataframe = pd.concat([dataframe, nuova_riga], ignore_index=True)

    # salviamo in un file csv in modalità append
    dataframe.to_csv(PATH_ESTR+'ambi.csv', mode='w', index=False, header=True)

"""     dataframe = dataframe.append({
        'Ambi': i, 
        'uscite': n_uscite, 
        'Spia10': spia10, 'rip10': rip10, 'tra10': tra10,
        'Spia20': spia20, 'rip20': rip20, 'tra20': tra20
        }, ignore_index=True) """

dataframe.head(100)

In [5]:
ambo = Lottomatica.Lotto()
ambo.previsioni('MI')
df = ambo.carica_estrazioni('MI')
ambo.carica_numero([87, 2], df, 10)
ambo.calcolo_trend()
#ambo.calcolo_zigzag(40)
ambo.stampa_statistihe()
ambo.print_trend()

i numeri da giocare su MI sono: [[87], [21], [72], [6], [61], [73], [27], [70], [33], [69], [86], [20], [8], [51], [80]]
Impossibile calcolare il trend
ruota: MI
numero: [87, 2]
n_uscite: 18
totale_estrazioni: 7085
ultima_frequenza: 393.61
ritardo_attuale: 11.0
ritardo_massimo: 1024.0
ultimo_ritardo: 352.0
ultima_scompensazione: -3337.841958123576
scompensazione_attuale: -3720.4519581235763
spia_num: 66
spia_rip: 18
spia_estr: 10
trend: nan
